<a href="https://colab.research.google.com/github/ayatoullah/.net-core/blob/master/cancelled_within_week_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()



Saving booking.csv to booking.csv


In [5]:
!pip install pandas pyarrow fastparquet


In [6]:
import pandas as pd
import random

df = pd.read_csv("booking.csv")
df['booking status'] = df['booking status'].apply(lambda x: 1 if x == 'Canceled' else 0)
df.head()



,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,lead time,market segment type,repeated,P-C,P-not-C,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,0
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,0
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,1
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,1
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,1


In [7]:
df = df.to_parquet("data.parquet", engine="pyarrow", index=False)


In [8]:
!pip install pyspark


In [11]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
df_spark = spark.read.parquet("data.parquet")

In [12]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["number of children","number of weekend nights","average price"],outputCol="features")
df_spark = assembler.transform(df_spark)
df_spark.head()

Row(Booking_ID='INN00001', number of adults=1, number of children=1, number of weekend nights=2, number of week nights=5, type of meal='Meal Plan 1', car parking space=0, room type='Room_Type 1', lead time=224, market segment type='Offline', repeated=0, P-C=0, P-not-C=0, average price=88.0, special requests=0, date of reservation='10/2/2015', booking status=0, features=DenseVector([1.0, 2.0, 88.0]))

In [15]:
from pyspark.sql import functions as F
df_spark = df_spark.withColumn("label", F.col("booking status"))
df_spark.show()

+----------+----------------+------------------+------------------------+---------------------+------------+-----------------+-----------+---------+-------------------+--------+---+-------+-------------+----------------+-------------------+--------------+----------------+-----+
|Booking_ID|number of adults|number of children|number of weekend nights|number of week nights|type of meal|car parking space|  room type|lead time|market segment type|repeated|P-C|P-not-C|average price|special requests|date of reservation|booking status|        features|label|
+----------+----------------+------------------+------------------------+---------------------+------------+-----------------+-----------+---------+-------------------+--------+---+-------+-------------+----------------+-------------------+--------------+----------------+-----+
|  INN00001|               1|                 1|                       2|                    5| Meal Plan 1|                0|Room_Type 1|      224|            Off